# Cell crop

This notebook uses both the outputs of cell_segmentation.ypynb and cell_area_analysis.ipynb to crop normalized cell images from the original images of the dataset

### Imports

In [9]:
import os
import sys
import json

import cv2 as cv
import pandas as pd
from tqdm import tqdm

sys.path.insert(0, "../../packages/python")
from models import cell_segmentation as segmentators

### Definitions

In [ ]:
sys.path.insert(0, "../../")
from config import MEDIA_PATH, TEMP_PATH

IMG_TARGET_SIDE = 200

# Dataset selection
DATASET = 'onion_cell_merged' # Change this to 'onion_cell_merged' if needed
DATASET_SECTION = 'test' # Change this to 'train', 'valida' or 'test' for onion_cell_merged dataset or empty for 'ina'

# Specific paths
IMAGES_PATH = os.path.join(MEDIA_PATH, f"images/{DATASET}/images/{DATASET_SECTION}")
CSV_PATH = os.path.join(MEDIA_PATH, f"cropped_images/{DATASET}/data/{DATASET_SECTION}")
CROPS_PATH = os.path.join(MEDIA_PATH, f"cropped_images/{DATASET}/images/{DATASET_SECTION}")
JSON_PATH = os.path.join(TEMP_PATH, 'datasets_area_data.json')

### List of elements to use

In [7]:
csvs = sorted(os.listdir(CSV_PATH)) #Paths to the csv of SAM detections of each image
images = sorted(os.listdir(IMAGES_PATH)) #full_images from where the crops are made
with open(JSON_PATH, 'r') as f: #json with the information of the filename of the images
    area_data = json.load(f)

### Dataset generation

In [8]:
resize_factor = IMG_TARGET_SIDE/area_data['INA']['lado_cuadrado']

if not os.path.exists(CROPS_PATH):
    os.makedirs(CROPS_PATH)


for image in tqdm(images):
 
    image_name, image_type =  image.split('.')
    image_group = image_name[0] if image_name[0].isalpha() else "INA" 
    image_side = area_data[image_group]['lado_cuadrado']
    image_resize_factor = int(resize_factor * image_side)

    img = cv.imread(os.path.join(IMAGES_PATH, image))

    df = pd.read_csv(os.path.join(CSV_PATH, f"{image_name}.csv"))
    df_bbox = df[df['image'] == image_name][['x', 'y', 'w', 'h', 'cell_id']]    

    for _, row in df_bbox.iterrows():
        cell_id = row['cell_id']    
        x, y, w, h = row['x'], row['y'], row['w'], row['h']
        x, y, w, h = segmentators.CellMaskGenerator.adjust_bbox(segmentators.CellMaskGenerator, x, y, w, h, image_resize_factor*image_resize_factor, img.shape[1], img.shape[0])

        crop = cv.resize(img[y:y+h, x:x+w], (IMG_TARGET_SIDE, IMG_TARGET_SIDE))
        output_path = os.path.join(CROPS_PATH, f"{image_name}_{cell_id}.{image_type}")
        cv.imwrite(output_path, crop)


                

100%|██████████| 63/63 [00:04<00:00, 14.00it/s]
